In [1]:
import os 
%pwd

'd:\\VS_code\\MLops\\11_End-to-End_Projects\\End-to-End_DS_Project_1\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\VS_code\\MLops\\11_End-to-End_Projects\\End-to-End_DS_Project_1'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path 
    model_name: str
    alpha: float 
    l1_ratio: float 
    target_column: str    

In [4]:
from src.dataScience.constants import *
from src.dataScience.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self, 
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH, 
            schema_file_path = SCHEMA_FILE_PATH
            ):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)   

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio= params.l1_ratio,
            target_column = schema.name
        )
        return model_trainer_config



In [6]:
import pandas as pd 
import os 
from src.dataScience import logger 
from sklearn.linear_model import ElasticNet
import joblib

In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column],axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        model = ElasticNet(alpha = self.config.alpha, 
                           l1_ratio= self.config.l1_ratio,
                           random_state=42)
        model.fit(X_train, y_train)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name) )
        

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-11-26 12:55:33,086: INFO: common: yaml file : config\config.yaml loaded successfully]
[2024-11-26 12:55:33,250: INFO: common: yaml file : params.yaml loaded successfully]
[2024-11-26 12:55:33,264: INFO: common: yaml file : schema.yaml loaded successfully]
[2024-11-26 12:55:33,312: INFO: common: Created directory at: artifacts]
[2024-11-26 12:55:33,394: INFO: common: Created directory at: artifacts/model_trainer]
